In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
from IPython.display import display
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

импорт библиотек

In [2]:
import warnings
warnings.filterwarnings('ignore')

отключение ворнингов

In [3]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

задание параметров таблиц

In [4]:
def find_id(x):
    xxx = x.find('span', class_="ty-control-group__item").text  
    return xxx

функция создания столбца с кодом товара

In [5]:
def find_availability(t):
    xxx = (t.find('a', class_="ty-control-group product-list-field cm-external-click")
           .text.replace('\ue924','').replace('\ue86c','').replace('\n',''))
    return xxx  

функция создания столбца с парсингом наличия товара

In [6]:
def find_title(t):
    xxx = t.find('a', class_="product-title").text
    return xxx

функция создающая столбец с названием товара

In [7]:
def find_price(t):
    xxx = t.find('span', class_="ty-price-num").text.replace(u'\xa0',u'')
    return xxx

функция создающая столбец с продажной ценой товара

In [8]:
def find_old_price(t):
    try:
        xxx = t.find('span', class_="ty-strike").text.replace(u'\xa0',u'').replace(u'руб',u'')
        return xxx
    except:
        return np.NaN

функция созжающая столбец с предидущей ценой для акционного товара

In [9]:
def find_url(t):
    xxx = t.find('a', class_="abt-single-image")['href']
    return xxx

Функция создающая столбец с урл адресом товара

In [10]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

Функция для проверки датафрейма(вид дублекаты пропуски)

In [11]:
def parsing_2(url):
    data = pd.DataFrame(columns=['url', 'id', 'date', 'availability', 'title', 'price', 'old_price'])
    for i in tqdm(url):        
        q = 1
        while q>0:        
            reqx = requests.get(i + str(q))
            soup = bs(reqx.text, 'html.parser')
            datax = pd.DataFrame({'url':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
            data = pd.concat([data, datax], ignore_index=True)
            if len(list(soup.findAll('div', class_='ypi-grid-list__item_body'))) > 0:
                q = q + 1
            else:
                q = 0
    data['id'] = data['url'].apply(find_id)
    data['date'] = pd.to_datetime('today').normalize()
    data['availability'] = data['url'].apply(find_availability)
    data['title'] = data['url'].apply(find_title)
    data['price'] = data['url'].apply(find_price)
    data['old_price'] = data['url'].apply(find_old_price)
    data['url'] = data['url'].apply(find_url)
    data['price'] = data['price'].astype(float)
    data.loc[data['old_price'] == '','old_price'] = np.nan
    data['old_price'] = data['old_price'].astype(float)
    return data

Функция предварительного парсинга сайта, которая проходит по всем страницам в урл адресах переменной `all_url` и создает датафрейм в котором храняться: урл адреса товаров, индентефикатор товаров, дата предварительного парсинга товара, наличия товара, названия товара, цены товара, старой цены для акционного товара.

# Парсинг цен кодов товара и url

In [12]:
all_url = ['https://hi-tech.md/televizory-i-elektronika/page-',
           'https://hi-tech.md/bytovaya-tehnika/page-',
           'https://hi-tech.md/kompyuternaya-tehnika/page-',
           'https://hi-tech.md/mebel-tekstil/kompyuternaya-mebel/ofisnye-kresla-i-stulya/komp.-kresla/page-',
           'https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/page-',
           'https://hi-tech.md/tovary-dlya-doma/page-'
          ]

переменная со списком ссылок на сайт для парсинга

In [13]:
df = parsing_2(all_url)

  0%|          | 0/6 [00:00<?, ?it/s]

предварительный парсинг

In [14]:
df = df.drop_duplicates().reset_index(drop=True)

удаление дублекатов

In [15]:
df.to_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)

создание резервной копии парсинга данных

In [16]:
df = pd.read_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

In [17]:
preprocessing(df)

таблица имеет следующий вид:


,url,id,date,availability,title,price,old_price
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr1220-01/,Т-000088228,2024-06-23,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2016-01/,Т-000088229,2024-06-23,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN
2,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aa4-01/,Т-000088221,2024-06-23,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN
3,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aaa4-01/,Т-000088222,2024-06-23,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN
4,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2025-01/,Т-000088230,2024-06-23,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN


****************************************************************************************************
таблица имеет 8734 сторок и 7 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
date               0
availability       0
title              0
price              0
old_price       6546
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
date             0.0
availability     0.0
title            0.0
price            0.0
old_price       74.9
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url              object
id               object
date             object
availability     object
title            object
price           float64
old_price       float64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8734 entries, 0 to 8733
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url           8734 non-null   object 
 1   id            8734 non-null   object 
 2   date          8734 non-null   object 
 3   availability  8734 non-null   object 
 4   title         8734 non-null   object 
 5   price         8734 non-null   float64
 6   old_price     2188 non-null   float64
dtypes: float64(2), object(5)
memory usage: 477.8+ KB


None

In [18]:
hai_group = pd.read_csv('csv/ahay_group.csv')

загрузка таблицы с группами товара

In [19]:
df = df.merge(hai_group, how='left', on='id')

обединение таблиц предварительного парсинга и таблицы групп товаров

In [20]:
df_new = df[(df['_category'].isna()) | (df['_subcategory'].isna()) | (df['_group'].isna()) | (df['_subgroup'].isna())]

создание тыблицы с товарами, у которых нет данных о группах

In [21]:
df = df[~((df['_category'].isna()) | (df['_subcategory'].isna()) | (df['_group'].isna()) | (df['_subgroup'].isna()))]

отчистка таблицы парсинга от пропусков

df = df.sample(30)

тестовое уменьшение таблици

In [22]:
def xxx(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    elem_2 = soup.findAll('div', class_='ty-breadcrumbs clearfix')
    name = soup.findAll('span', class_='ty-breadcrumbs__current')
            
    try:
        names = name[0].text
        name_group = elem_2[0].text.replace('/'+str(names),'').replace('\n','')
        err = 'err'
    except:
        names = 'Noname'
        name_group = 'Noname/Noname/Noname/Noname/Noname/Noname'
        err = 'Noname_err'
    try:
        len_name = len((elem_2[0].text.replace('/'+str(names),'').replace('\n','')).split('/'))
    except:
        len_name = 1
        
    try:
        if len_name <= 1:
            d_14 = 'err1'
            d_15 = 'err1'
            d_16 = 'err1'
            d_17 = 'err1'
        elif len_name <= 2:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_17 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()      
        elif len_name <= 3:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_17 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
        elif len_name == 4:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower() 
            d_17 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower()
        elif len_name > 4:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower() 
            d_17 = 'H_'+(name_group.split("/")[4]).replace(',','').replace(' ','_').replace('-','_').lower()
    except:
        d_14 = 'err0'
        d_15 = 'err0'
        d_16 = 'err0'
        d_17 = 'err0'
        
    return pd.Series([(d_14), (d_15), (d_16), (d_17)])

Функция основного парсинга, которая для каждой строки по урл адресу парсит и находит наличие товара в магахинах а также группу товара, категорию товара, подкатегорию товата и количество подкатегорий товара

# Парсинг групп товара

In [23]:
tqdm.pandas() 
df_new[['_category','_subcategory','_group','_subgroup']] = df_new['url'].progress_apply(xxx)

  0%|          | 0/2053 [00:00<?, ?it/s]

Основной парсинг

In [24]:
data = df_new[~((df_new['_category']=='err1') | (df_new['_category']=='err0'))].reset_index(drop=True)
data = data[['id', '_category','_subcategory','_group', '_subgroup']]
data = data.sort_values(['_category','_subcategory', '_group', '_subgroup'])

создание таблици с группами нового товара

In [25]:
hai_group = pd.concat([hai_group, data], ignore_index=True).sort_values(['_category','_subcategory', '_group', '_subgroup']).reset_index(drop=True)
hai_group.to_csv('csv/ahay_group.csv', index=False)  

добавление в таблицу с группами товара новых данных

In [26]:
df = pd.concat([df, df_new], ignore_index=True).sort_values(['_category','_subcategory', '_group', '_subgroup']).reset_index(drop=True)

объединение таблици со старыми и новыми товарами

In [27]:
df._category.unique()

array(['H_акция', 'H_бытовая_техника', 'H_компьютерная_техника',
       'H_мебель_текстиль', 'H_стройка_и_ремонт',
       'H_телевизоры_и_электроника', 'H_товар_без_акции',
       'H_товар_по_акции', 'H_товары_для_дома'], dtype=object)

In [28]:
df._subcategory.unique()

array(['H_рассрочка_12_месяцев___без_переплаты!',
       'H_аксессуары_для_мелкой_бытовой_техники',
       'H_климатическая_техника', 'H_крупная_техника_для_кухни',
       'H_мелкая_техника_для_кухни', 'H_минимойки',
       'H_приготовление_напитков', 'H_приготовление_пищи',
       'H_пылесосы_и_аксессуары', 'H_стиральные_и_сушильные_машины',
       'H_техника_karcher', 'H_техника_для_здоровья',
       'H_техника_для_красоты', 'H_товары_для_ухода_за_одеждой',
       'H_установка_бытовой_техники', 'H_apple', 'H_аксессуары',
       'H_видеотехника', 'H_готовые_решения', 'H_игровые_приставки',
       'H_комплектующие', 'H_крепления', 'H_мониторы', 'H_мультимедиа',
       'H_накопители', 'H_оргтехника', 'H_периферийные_устройства',
       'H_проекторы_и_экраны', 'H_расходные_материалы',
       'H_сетевое_оборудование', 'H_умный_дом_с_яндекс',
       'H_электропитание', 'H_мебель_для_офиса', 'H_техника_для_дома',
       'H_электроинструменты', 'H_аудиосистемы',
       'H_батарейки_и_зарядны

In [29]:
df._group.unique()

array(['H_рассрочка_12_месяцев___без_переплаты!',
       'H_аксессуары_для_зубных_щеток',
       'H_аксессуары_для_электрических_бритв', 'H_бойлеры',
       'H_вентиляторы', 'H_комплектующие_для_климатической_техники',
       'H_кондиционеры', 'H_кондиционеры_кассетные',
       'H_кондиционеры_мульти_сплит', 'H_кондиционеры_промышленные',
       'H_котлы_газовые', 'H_обогреватели', 'H_отопительные_печи',
       'H_увлажнители_комплексы', 'H_варочные_поверхности', 'H_вытяжки',
       'H_духовые_шкафы', 'H_кронштейны_для_микроволновых_печей',
       'H_микроволновые_печи', 'H_морозильники_и_лари',
       'H_плиты_газовые_и_электрические', 'H_посудомоечные_машины',
       'H_холодильники', 'H_электропечи_и_духовки', 'H_блендеры',
       'H_вакууматоры', 'H_весы_кухонные', 'H_генераторы_льда',
       'H_кухонные_комбайны', 'H_ломтерезки', 'H_миксеры',
       'H_соковыжималки', 'H_соковыжималки_для_цитрусов', 'H_тостеры',
       'H_чопперы', 'H_электромясорубки', 'H_минимойки', 'H_кофе',
  

просмотр категорий, подкатегорий и групп товаров

In [30]:
df[df['_category']=='err1'].id.count()

0

In [31]:
df[df['_category']=='err0'].id.count()

0

In [32]:
df[df['_category']=='err1']

,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup


In [33]:
df[df['_category']=='err0']

,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup


просмотр товаров с ошибками

In [34]:
preprocessing(df)

таблица имеет следующий вид:


,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup
0,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/kondicionery/kondicioner-electrolux-eacs-07hat-n321y/,Т-000078544,2024-06-23,В наличии,кондиционер Electrolux EACS-07HAT/N3_21Y,3999.0,NaN,H_акция,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,2024-06-23,В наличии,насадка д/Зубная щетка Panasonic EW0940W830 2шт,34.0,68.0,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток
2,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb10-2k-kids-frozen-1sht/,Т-000021198,2024-06-23,В наличии,насадка д/Зубная щетка Braun EB10 Kids Frozen (1шт),94.0,NaN,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток
3,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb18-3d-white-1sht/,Т-000068433,2024-06-23,В наличии,насадка д/Зубная щетка Braun EB18 3D White (1шт),94.0,NaN,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток
4,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb20rb-10-1sht/,Т-000068766,2024-06-23,В наличии,насадка д/Зубная щетка Braun EB20 Precision Clean (1шт),94.0,NaN,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток


****************************************************************************************************
таблица имеет 8734 сторок и 11 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
date               0
availability       0
title              0
price              0
old_price       6546
_category          0
_subcategory       0
_group             0
_subgroup          0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
date             0.0
availability     0.0
title            0.0
price            0.0
old_price       74.9
_category        0.0
_subcategory     0.0
_group           0.0
_subgroup        0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url              object
id               object
date             object
availability     object
title            object
price           float64
old_price       float64
_category        object
_subcategory     object
_group           object
_subgroup        object
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8734 entries, 0 to 8733
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url           8734 non-null   object 
 1   id            8734 non-null   object 
 2   date          8734 non-null   object 
 3   availability  8734 non-null   object 
 4   title         8734 non-null   object 
 5   price         8734 non-null   float64
 6   old_price     2188 non-null   float64
 7   _category     8734 non-null   object 
 8   _subcategory  8734 non-null   object 
 9   _group        8734 non-null   object 
 10  _subgroup     8734 non-null   object 
dtypes: float64(2), object(9)
memory usage:

None

просмотр данных о таблице

In [35]:
df.to_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)

создание резервной копии парсинга данных

In [36]:
df = pd.read_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

загрузка резервной копии парсинга данных

In [37]:
df.head(2)

,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup
0,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/kondicionery/kondicioner-electrolux-eacs-07hat-n321y/,Т-000078544,2024-06-23,В наличии,кондиционер Electrolux EACS-07HAT/N3_21Y,3999.0,NaN,H_акция,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,2024-06-23,В наличии,насадка д/Зубная щетка Panasonic EW0940W830 2шт,34.0,68.0,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток


In [38]:
df['error_group'] = 0
df.loc[df['_subgroup'].isna(), 'error_group'] = 1
df.loc[df['_subgroup']=='err0', 'error_group'] = 1
df.loc[df['_subgroup']=='err1', 'error_group'] = 1

Создание столбцов с флагом ошибки для товаров, для которых нет данных по остаткам хотябы в одном магазине

In [39]:
df.loc[df['error_group']==1 , ['_category',
       '_subcategory', '_group', '_subgroup']] = 'H_no_group'

удаление наличия товара в магазинах и наименовании группы если у товара есть ошибки в этих данны

Изменение типов данных столбцов наличия товара в определенном магазине на тип данных `float`  и создания столбцов по наличию товара в магазинах городов

In [40]:
df = df[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'title',
               'date',   'price', 'old_price', 'availability', 'error_group', ]]

изменение порядка располажения столбцов

In [41]:
preprocessing(df)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,_subgroup,title,date,price,old_price,availability,error_group
0,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/kondicionery/kondicioner-electrolux-eacs-07hat-n321y/,Т-000078544,H_акция,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,кондиционер Electrolux EACS-07HAT/N3_21Y,2024-06-23,3999.0,NaN,В наличии,0
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Panasonic EW0940W830 2шт,2024-06-23,34.0,68.0,В наличии,0
2,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb10-2k-kids-frozen-1sht/,Т-000021198,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Braun EB10 Kids Frozen (1шт),2024-06-23,94.0,NaN,В наличии,0
3,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb18-3d-white-1sht/,Т-000068433,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Braun EB18 3D White (1шт),2024-06-23,94.0,NaN,В наличии,0
4,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb20rb-10-1sht/,Т-000068766,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Braun EB20 Precision Clean (1шт),2024-06-23,94.0,NaN,В наличии,0


****************************************************************************************************
таблица имеет 8734 сторок и 12 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
_category          0
_subcategory       0
_group             0
_subgroup          0
title              0
date               0
price              0
old_price       6546
availability       0
error_group        0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
_category        0.0
_subcategory     0.0
_group           0.0
_subgroup        0.0
title            0.0
date             0.0
price            0.0
old_price       74.9
availability     0.0
error_group      0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url              object
id               object
_category        object
_subcategory     object
_group           object
_subgroup        object
title            object
date             object
price           float64
old_price       float64
availability     object
error_group       int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8734 entries, 0 to 8733
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url           8734 non-null   object 
 1   id            8734 non-null   object 
 2   _category     8734 non-null   object 
 3   _subcategory  8734 non-null   object 
 4   _group        8734 non-null   object 
 5   _subgroup     8734 non-null   object 
 6   title         8734 non-null   object 
 7   date          8734 non-null   object 
 8   price         8734 non-null   float64
 9   old_price     2188 non-null   float64
 10  availability  8734 non-null   object 
 11  error_group   8734 non-null   int64  


None

просмотр данных после парсинга

# Группировка товара

- разобраться с креслами в ногруп

In [42]:
df.loc[df['_category']=='H_товар_без_акции', '_category'] = 'H_no_group'
df.loc[df['_category']=='H_товар_по_акции', '_category'] = 'H_no_group'
df.loc[df['_subcategory']=='H_мультимедиа', '_subcategory'] = 'H_аудиосистемы'
df.loc[df['_group']=='H_акустические_системы', '_group'] = 'H_акустика'
df.loc[df['_group']=='H_музыкальная_техника', '_group'] = 'H_акустика'
df.loc[df['_subcategory']=='H_умный_дом_с_яндекс','_subcategory'] = 'H_аудиосистемы'
df.loc[df['_group']=='H_яндекс_станции', '_group'] = 'H_акустика'
df.loc[df['_group']=='H_airpods', '_subcategory'] = 'H_аудиосистемы'
df.loc[df['_group']=='H_airpods', '_group'] = 'H_наушники'
df.loc[df['_group']=='H_apple_watch', '_group'] = 'H_часы_и_браслеты'
df.loc[df['_group']=='H_iphone', '_group'] = 'H_смартфоны'
df.loc[df['_group']=='H_ipad', '_group'] = 'H_планшеты'
df.loc[df['_group']=='H_macbook', '_group'] = 'H_ноутбуки'
df.loc[df['_group']=='H_imac', '_group'] = 'H_моноблоки'
df.loc[df['_group']=='H_часы_и_браслеты', '_subcategory'] = 'H_гаджеты'
df.loc[df['_group']=='H_смартфоны', '_subcategory'] = 'H_смартфоны_и_аксессуары'
df.loc[df['_group']=='H_планшеты', '_subcategory'] = 'H_готовые_решения'
df.loc[df['_group']=='H_ноутбуки', '_subcategory'] = 'H_готовые_решения'
df.loc[df['_group']=='H_моноблоки', '_subcategory'] = 'H_готовые_решения'
df.loc[df['_subcategory']=='H_стиральные_и_сушильные_машины', '_group'] = 'H_стиральные_машины'
df.loc[df['_subgroup']=='H_сушильные_машины', '_group'] = 'H_сушильные_машины'
df.loc[df['_subgroup']=='H_наклейки', '_group'] = 'H_наклейки'
df.loc[df['_subgroup']=='H_подставки', '_group'] = 'H_подставки'
df.loc[df['_subgroup']=='H_замки', '_group'] = 'H_замки'
df.loc[df['_subgroup']=='H_usb_hub', '_group'] = 'H_usb_hub'
df.loc[df['_subgroup']=='H_сумки_и_чехлы', '_group'] = 'H_сумки_и_чехлы'
df.loc[df['_subgroup']=='H_защитные_стекла', '_group'] = 'H_защитные_стекла'
df.loc[df['_subcategory']=='H_мониторы','_group'] = 'H_мониторы'

In [43]:
for i in df['_category'].unique():
    t = df[df['_category']==i]
    print(i)
    print(t['_subcategory'].unique())
    print('-'*100)

H_акция
['H_рассрочка_12_месяцев___без_переплаты!']
----------------------------------------------------------------------------------------------------
H_бытовая_техника
['H_аксессуары_для_мелкой_бытовой_техники' 'H_климатическая_техника'
 'H_крупная_техника_для_кухни' 'H_мелкая_техника_для_кухни' 'H_минимойки'
 'H_приготовление_напитков' 'H_приготовление_пищи'
 'H_пылесосы_и_аксессуары' 'H_стиральные_и_сушильные_машины'
 'H_техника_karcher' 'H_техника_для_здоровья' 'H_техника_для_красоты'
 'H_товары_для_ухода_за_одеждой' 'H_установка_бытовой_техники']
----------------------------------------------------------------------------------------------------
H_компьютерная_техника
['H_аудиосистемы' 'H_гаджеты' 'H_готовые_решения'
 'H_смартфоны_и_аксессуары' 'H_apple' 'H_аксессуары' 'H_видеотехника'
 'H_игровые_приставки' 'H_комплектующие' 'H_крепления' 'H_мониторы'
 'H_накопители' 'H_оргтехника' 'H_периферийные_устройства'
 'H_проекторы_и_экраны' 'H_расходные_материалы' 'H_сетевое_оборудован

In [44]:
for i in df['_subcategory'].unique():
    t = df[df['_subcategory']==i]
    print(i)
    print(t['_group'].unique())
    print('-'*100)

H_рассрочка_12_месяцев___без_переплаты!
['H_рассрочка_12_месяцев___без_переплаты!']
----------------------------------------------------------------------------------------------------
H_аксессуары_для_мелкой_бытовой_техники
['H_аксессуары_для_зубных_щеток' 'H_аксессуары_для_электрических_бритв']
----------------------------------------------------------------------------------------------------
H_климатическая_техника
['H_бойлеры' 'H_вентиляторы' 'H_комплектующие_для_климатической_техники'
 'H_кондиционеры' 'H_кондиционеры_кассетные' 'H_кондиционеры_мульти_сплит'
 'H_кондиционеры_промышленные' 'H_котлы_газовые' 'H_обогреватели'
 'H_отопительные_печи' 'H_увлажнители_комплексы']
----------------------------------------------------------------------------------------------------
H_крупная_техника_для_кухни
['H_варочные_поверхности' 'H_вытяжки' 'H_духовые_шкафы'
 'H_кронштейны_для_микроволновых_печей' 'H_микроволновые_печи'
 'H_морозильники_и_лари' 'H_плиты_газовые_и_электрические'
 'H_по

просмотр групп товаров в подкатегориях

In [45]:
for i in df['_group'].unique():
    t = df[df['_group']==i]
    print(i)
    print(t['_subgroup'].unique())
    print('-'*100)

H_рассрочка_12_месяцев___без_переплаты!
['H_рассрочка_12_месяцев___без_переплаты!']
----------------------------------------------------------------------------------------------------
H_аксессуары_для_зубных_щеток
['H_аксессуары_для_зубных_щеток']
----------------------------------------------------------------------------------------------------
H_аксессуары_для_электрических_бритв
['H_аксессуары_для_электрических_бритв']
----------------------------------------------------------------------------------------------------
H_бойлеры
['H_бойлеры']
----------------------------------------------------------------------------------------------------
H_вентиляторы
['H_вентиляторы']
----------------------------------------------------------------------------------------------------
H_комплектующие_для_климатической_техники
['H_комплектующие_для_климатической_техники']
----------------------------------------------------------------------------------------------------
H_кондиционеры
['H_конди

['H_отпариватели_для_одежды']
----------------------------------------------------------------------------------------------------
H_парогенераторы
['H_парогенераторы']
----------------------------------------------------------------------------------------------------
H_сушилки_для_белья
['H_сушилки_для_белья']
----------------------------------------------------------------------------------------------------
H_утюги
['H_утюги']
----------------------------------------------------------------------------------------------------
H_швейные_машины
['H_швейные_машины']
----------------------------------------------------------------------------------------------------
H_установка_бытовой_техники
['H_установка_бытовой_техники']
----------------------------------------------------------------------------------------------------
H_наушники
['H_airpods' 'H_вкладыши' 'H_накладные' 'H_наушники']
---------------------------------------------------------------------------------------------------

H_телевизоры
['H_телевизоры']
----------------------------------------------------------------------------------------------------
H_телевизоры_samsung
['H_телевизоры_samsung']
----------------------------------------------------------------------------------------------------
H_телефоны_dect_и_проводные
['H_телефоны_dect_и_проводные']
----------------------------------------------------------------------------------------------------
H_товар_без_акции
['H_товар_без_акции']
----------------------------------------------------------------------------------------------------
H_товар_по_акции
['H_товар_по_акции']
----------------------------------------------------------------------------------------------------
H_средства_для_посудомоечных_машин
['H_средства_для_посудомоечных_машин']
----------------------------------------------------------------------------------------------------
H_средства_для_стирки
['H_средства_для_стирки']
----------------------------------------------------------

In [46]:
df._category.unique()

array(['H_акция', 'H_бытовая_техника', 'H_компьютерная_техника',
       'H_мебель_текстиль', 'H_стройка_и_ремонт',
       'H_телевизоры_и_электроника', 'H_no_group', 'H_товары_для_дома'],
      dtype=object)

In [47]:
df._subcategory.unique()

array(['H_рассрочка_12_месяцев___без_переплаты!',
       'H_аксессуары_для_мелкой_бытовой_техники',
       'H_климатическая_техника', 'H_крупная_техника_для_кухни',
       'H_мелкая_техника_для_кухни', 'H_минимойки',
       'H_приготовление_напитков', 'H_приготовление_пищи',
       'H_пылесосы_и_аксессуары', 'H_стиральные_и_сушильные_машины',
       'H_техника_karcher', 'H_техника_для_здоровья',
       'H_техника_для_красоты', 'H_товары_для_ухода_за_одеждой',
       'H_установка_бытовой_техники', 'H_аудиосистемы', 'H_гаджеты',
       'H_готовые_решения', 'H_смартфоны_и_аксессуары', 'H_apple',
       'H_аксессуары', 'H_видеотехника', 'H_игровые_приставки',
       'H_комплектующие', 'H_крепления', 'H_мониторы', 'H_накопители',
       'H_оргтехника', 'H_периферийные_устройства',
       'H_проекторы_и_экраны', 'H_расходные_материалы',
       'H_сетевое_оборудование', 'H_электропитание', 'H_мебель_для_офиса',
       'H_техника_для_дома', 'H_электроинструменты',
       'H_батарейки_и_зарядны

In [48]:
df._group.unique()

array(['H_рассрочка_12_месяцев___без_переплаты!',
       'H_аксессуары_для_зубных_щеток',
       'H_аксессуары_для_электрических_бритв', 'H_бойлеры',
       'H_вентиляторы', 'H_комплектующие_для_климатической_техники',
       'H_кондиционеры', 'H_кондиционеры_кассетные',
       'H_кондиционеры_мульти_сплит', 'H_кондиционеры_промышленные',
       'H_котлы_газовые', 'H_обогреватели', 'H_отопительные_печи',
       'H_увлажнители_комплексы', 'H_варочные_поверхности', 'H_вытяжки',
       'H_духовые_шкафы', 'H_кронштейны_для_микроволновых_печей',
       'H_микроволновые_печи', 'H_морозильники_и_лари',
       'H_плиты_газовые_и_электрические', 'H_посудомоечные_машины',
       'H_холодильники', 'H_электропечи_и_духовки', 'H_блендеры',
       'H_вакууматоры', 'H_весы_кухонные', 'H_генераторы_льда',
       'H_кухонные_комбайны', 'H_ломтерезки', 'H_миксеры',
       'H_соковыжималки', 'H_соковыжималки_для_цитрусов', 'H_тостеры',
       'H_чопперы', 'H_электромясорубки', 'H_минимойки', 'H_кофе',
  

In [49]:
df._subgroup.unique()

array(['H_рассрочка_12_месяцев___без_переплаты!',
       'H_аксессуары_для_зубных_щеток',
       'H_аксессуары_для_электрических_бритв', 'H_бойлеры',
       'H_вентиляторы', 'H_комплектующие_для_климатической_техники',
       'H_кондиционеры', 'H_кондиционеры_кассетные',
       'H_кондиционеры_мульти_сплит', 'H_кондиционеры_промышленные',
       'H_котлы_газовые', 'H_обогреватели', 'H_отопительные_печи',
       'H_увлажнители_комплексы', 'H_варочные_поверхности', 'H_вытяжки',
       'H_духовые_шкафы', 'H_кронштейны_для_микроволновых_печей',
       'H_микроволновые_печи', 'H_морозильники', 'H_морозильные_лари',
       'H_газовые_плиты', 'H_комбинированные_плиты',
       'H_плиты_газовые_и_электрические', 'H_электрические_плиты',
       'H_посудомоечные_машины', 'H_side_by_side_и_french_door',
       'H_винные_шкафы', 'H_встраиваемые_холодильники',
       'H_однодверные_холодильники', 'H_с_верхней_морозильной_камерой',
       'H_с_нижней_морозильной_камерой', 'H_холодильники',
       'H_

# Подготовка анализа

In [50]:
df.head(2)

,url,id,_category,_subcategory,_group,_subgroup,title,date,price,old_price,availability,error_group
0,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/kondicionery/kondicioner-electrolux-eacs-07hat-n321y/,Т-000078544,H_акция,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,кондиционер Electrolux EACS-07HAT/N3_21Y,2024-06-23,3999.0,NaN,В наличии,0
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Panasonic EW0940W830 2шт,2024-06-23,34.0,68.0,В наличии,0


In [51]:
brend = pd.read_excel('csv/brend.xlsx')

In [52]:
def lower(x):
    return x.lower()
brend['names'] = brend['name'].apply(lower)
brend_up = brend.name.unique()
brend_low = brend.names.unique()

Создаем список брендов

In [53]:
def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'
tqdm.pandas()
df['brend'] = df['title'].progress_apply(brends)

  0%|          | 0/8734 [00:00<?, ?it/s]

создаем столбец с брендами

In [54]:
df['action_sale'] = 0
df.loc[df['old_price'] > 0,'action_sale'] = 1

Создаем столбец с флагом акции

In [55]:
df['sale'] = (100 - ((df['price']*100)/df['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [56]:
def count_group(x):
    return df[df['_group']==x]['id'].count()    
tqdm.pandas()
df['count_group'] = df['_group'].progress_apply(count_group)

  0%|          | 0/8734 [00:00<?, ?it/s]

создание столбца с количеством товара в группе

In [57]:
def count_group_presence(x):
    return df[(df['availability']=='В наличии')&(df['_group']==x)]['id'].count()   
tqdm.pandas()
df['activ_count_group'] = df['_group'].progress_apply(count_group_presence)

  0%|          | 0/8734 [00:00<?, ?it/s]

создание столбца с количеством товара, который есть в наличии, в группе

In [58]:
def dif_price(x):    
    max_price = df[(df['availability']=='В наличии')&(df['_group']==x)]['price'].max()
    min_price = df[(df['availability']=='В наличии')&(df['_group']==x)]['price'].min() 
    return max_price / min_price
tqdm.pandas()
df['dif_price'] = df['_group'].progress_apply(dif_price)

  0%|          | 0/8734 [00:00<?, ?it/s]

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [59]:
df['group_prise'] = df.groupby('_group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", 'дорогой', "элитный"]))

создание столбца с ценовой категорией товара

In [60]:
df = df[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'brend', 
         'title', 'date', 'price', 'old_price', 'action_sale', 'sale', 'group_prise', 'dif_price',
         'availability', 'count_group', 'activ_count_group','error_group']]

In [61]:
df.drop('_subgroup', axis= 1 , inplace= True )

In [63]:
df['tir'] = 0

In [64]:
df.to_csv('csv/hi_tech_final_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False) 

# Аналитика акционных товаров

In [ ]:
df.head(3)

In [ ]:
df.groupby('_group')['price'].mean().reset_index()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
df[df['_group']=='H_пылесосы'].head(2)

In [ ]:
separator = ['_group','brend','group_prise']

In [ ]:
for x in separator:
    for i in df['_subcategory'].unique():        
        plt.figure(figsize=(16,9))
        sns.barplot(df[df['_subcategory']==i].groupby([x ,'action_sale'])['id'].count().sort_values(ascending=False).reset_index(),
                       x=x, y='id', hue='action_sale')
        plt.xticks(rotation=20)
        plt.title(f'количество товаров разных {x} в подкатегории {i} распределенные по наличию акции ')
        plt.xlabel(f' {x}')
        plt.ylabel('количество товара')
        plt.show()

In [ ]:
df[df['_subcategory']=='пылесосы_и_аксессуары'].groupby(['_group' ])['action_sale'].mean().reset_index()

In [ ]:
for x in separator:
    for i in df['_subcategory'].unique():        
        plt.figure(figsize=(16,9))
        sns.barplot(df[df['_subcategory']==i].groupby(x)['action_sale'].mean().sort_values(ascending=False).reset_index(),
                       x=x, y='action_sale', color='blue')
        plt.xticks(rotation=20)
        plt.title(f'количество товаров разных {x} в подкатегории {i} ')
        plt.xlabel(f' {x}')
        plt.ylabel('доля акционного товара')
        plt.show()